## Visualizing Experimental Results

In [10]:
import partitura as pt
import parangonar as pg
import pandas as pd
import os
import numpy as np

In [32]:
par_path = os.path.join(os.getcwd(), "artifacts")
follower = "oltw"
name = "chopin_op10_No11"
gt_path = os.path.join(
    os.path.normpath("C:\\Users\\melki\\Desktop\\JKU\\data\\accompanion_experiment\\match"), name + ".match")

## Evaluate Alignment

This part of the experiment evaluates the matched notes by the score follower to the ground truth alignment.

In [33]:
# Load the alignment file
a = pd.read_csv(os.path.join(par_path, f"{name}_{follower}_alignment.csv"), sep=",")
a.head()

,idx,matchtype,partid,ppartid
0,0,0,n1-1,0
1,1,0,n21-1,1
2,2,0,n5-1,2
3,3,0,n4-1,3
4,4,0,n3-1,4


In [34]:
# Structure alignment as accepted input for parangonar fscore
pred_alignment = [{"label": ("match" if row[1].matchtype==0 else "insertion"), "score_id": row[1].partid, "performance_id": row[1].ppartid[4:]} for row in a.iterrows()]

# Load Ground Truth Alignment
gt_ppart, gt_alignment, score = pt.load_match(gt_path, create_score=True)

fmeasure = pg.fscore_alignments(pred_alignment, gt_alignment, types=["match"])
print(fmeasure)

TypeError: 'int' object is not subscriptable

## Time Delay Evaluation

This experiment evaluates the time delay between the expected onsets and the performed onsets.
The score follower predicts a score position and a beat period, which we can use to compute the expected onset in seconds and compare it to the performed onset in seconds.

In [14]:
time_delays = pd.read_csv(os.path.join(par_path, name + "_time_delays.csv"), index_col=0, sep=",")
time_delays.head()

,Solo Performance Onset,Beat Period
Solo Score Onset,,
-3.0,0.501469,0.500000
-2.5,0.877298,0.500000
-2.0,1.175111,0.700221
-1.5,1.457543,0.689483
-1.0,1.787076,0.682160


In [15]:
# # Normalize time delay to get only positive beats
# time_delays.index = time_delays.index - time_delays.index.min() if time_delays.index.min() < 0 else time_delays.index
# # Get the relative performed positions
# ioi_perf = np.r_[0, np.diff(time_delays.index.to_numpy())] * time_delays["Beat Period"].to_numpy()
# # Get the cumlative sum to get seconds
# pred_ptime = np.cumsum(ioi_perf)
# # Get the mean difference / time delay
# tdiff = np.abs(pred_ptime - time_delays["Solo Performance Onset"].to_numpy()).mean()
# print(tdiff)

In [16]:
pnote_array = gt_ppart.note_array()
note_array = score.note_array()

pred_beats = time_delays.index.to_numpy()
s_beat_delays = list()
for idx, p_onset in enumerate(time_delays["Solo Performance Onset"].to_numpy()):
    p_id = pnote_array[np.argmin(np.abs(pnote_array["onset_sec"] - p_onset))]["id"]
    for a in gt_alignment:
        if a["label"] == "match":
            if a["performance_id"] == p_id:
                onset = note_array[note_array["id"]==a["score_id"]]["onset_beat"]
                s_beat_delays.append(abs(pred_beats[idx] - onset))
                break

In [29]:
s_beat_delays = np.array(s_beat_delays)
print("Average delay in beats for piece {} using the {} follower is : \n{} beats".format(name, follower, s_beat_delays.mean()))

Average delay in beats for piece chopin_op09_No1 using the hmm follower is : 
1.7857807874679565 beats


In [18]:
pnote_array = gt_ppart.note_array()
note_array = score.note_array()

pred_onset = time_delays["Solo Performance Onset"].to_numpy()
p_time_delays = list()
for idx, s_onset in enumerate(time_delays.index.to_numpy()):
    s_idx = note_array[note_array["onset_beat"] == s_onset]["id"]
    tmp = []
    for s_id in s_idx:
        for a in gt_alignment:
            if a["label"] == "match":
                if a["score_id"] == s_id:
                    onset = pnote_array[pnote_array["id"]==a["performance_id"]]["onset_sec"]
                    tmp.append(abs(pred_onset[idx] - onset))
    if tmp:
        p_time_delays.append(min(tmp))

In [31]:
p_time_delays = np.array(p_time_delays)
print("Average delay in seconds for piece {} using the {} follower is : \n{} seconds".format(name, follower, p_time_delays.mean()))

Average delay in seconds for piece chopin_op09_No1 using the hmm follower is : 
1.126117467880249 seconds


## Exploring MIDI Degradation

This part of the code explores ways to degrade a midi performance to create increasingly difficult performances to follow.

In [1]:
import pandas as pd
import numpy as np
import partitura as pt
from numpy.lib.recfunctions import unstructured_to_structured
from mdtk.degradations import (
    pitch_shift,
    time_shift,
    onset_shift,
    offset_shift,
    remove_note,
    add_note,
)

ModuleNotFoundError: No module named 'mdtk'

In [2]:
piece_path = os.path.normpath("/home/manos/Desktop/JKU/data/accompanion_experiment/midi/chopin_op09_no1.mid")
performance = pt.load_performance(piece_path)
piece_name = os.path.splitext(os.path.basename(piece_path))[0]
note_array = performance.note_array()
ppq = performance[0].ppq
mpq = performance[0].mpq
odf = pd.DataFrame(note_array)
odf.head()

Error loading score as midi:
[Errno 2] No such file or directory: '/home/manos/Desktop/JKU/data/midi/inventions/BachJS-BWV772.mid'
Error loading score as match:
Filename does not exist


NotSupportedFormatError: 

In [37]:
def degradation_from_note_array(note_array, mpq, ppq, min_pitch=36, max_pitch=110, min_shift=10, max_shift=1_000):
    """
    Degrade Midi from note_array.

    Parameters
    ----------
    note_array: numpy structured array
        The performed part array
    mpq: int
    ppq: int
    min_pitch: int (optional)
        degrade pitch by randomly moving from min_pitch upwards
    max_pitch: int (optional)
        degrade pitch by randomly moving from max_pitch downwards
    min_shift : int (optional)
        Move random onsets and offsets at least 10 msec.
    max_shift: int (optinoal)
        Move random onsets and offsets at most 1 sec.

    Returns
    -------
    ppart : partitura.performance.Part
        A performed part created from note array.
    """
    odf = pd.DataFrame(
        {
            "onset": note_array["onset_tick"],
            "pitch": note_array["pitch"],
            "dur": note_array["duration_tick"],
            "velocity": note_array["velocity"],
            "track": note_array["track"]
        }
    )
    min_pitch = min_pitch  # cant shift lower than 36 (default)
    max_pitch = max_pitch # cant shift higher than 110 (default)
    distribution = [1, 0, 1, 0, 1, 0, 1]  # only shift min 2nd or maj 3rd up/down
    df = pitch_shift(
        odf,
        min_pitch=min_pitch,
        max_pitch=max_pitch,
        distribution=distribution,
        seed=42
    )


    df = onset_shift(
        df,
        min_shift=min_shift, # dont shift less than .01s (default)
        max_shift=max_shift, # dont shift more than 1s (default)
    )

    df = offset_shift(
        df,
        min_shift,
        max_shift
    )

    df["id"] = odf["id"]
    df["channel"] = odf["channel"]
    df["onset_sec"] = pt.utils.music.midi_ticks_to_seconds(df["onset"].to_numpy(), mpq=mpq, ppq=ppq)
    df["duration_sec"] = pt.utils.music.midi_ticks_to_seconds(df["dur"].to_numpy(), mpq=mpq, ppq=ppq)

    records = df.to_records(index=False)
    new_note_array = np.array(records, dtype = records.dtype.descr)
    ppart = pt.performance.PerformedPart.from_note_array(new_note_array)
    return ppart

In [43]:


deg_excerpt_1 = pitch_shift(
    df,
    min_pitch=min_pitch,
    max_pitch=max_pitch,
    distribution=distribution,
    seed=42
)
deg_excerpt_1["id"] = odf["id"]
deg_excerpt_1["channel"] = odf["channel"]
deg_excerpt_1["onset_sec"] = pt.utils.music.midi_ticks_to_seconds(deg_excerpt_1["onset"].to_numpy(), mpq=mpq, ppq=ppq)
deg_excerpt_1["duration_sec"] = pt.utils.music.midi_ticks_to_seconds(deg_excerpt_1["dur"].to_numpy(), mpq=mpq, ppq=ppq)
deg_excerpt_1.head()

,onset,track,pitch,dur,velocity,id,channel,onset_sec,duration_sec
0,2,0,60,2,64,P00_n0,1,0.125,0.125
1,4,0,62,2,64,P00_n1,1,0.250,0.125
2,6,0,64,2,64,P00_n2,1,0.375,0.125
3,8,0,65,2,64,P00_n3,1,0.500,0.125
4,10,0,62,2,64,P00_n4,1,0.625,0.125


In [ ]:
min_shift = 10  # dont shift less than .01s
max_shift = 1_000   # dont shift more than 1s

deg_excerpt_2 = onset_shift(
    df,
    min_shift=min_pitch,
    max_shift=max_pitch,
)

deg_excerpt_2 = offset_shift(
    deg_excerpt_2,
    min_shift,
    max_shift
)

deg_excerpt_2["id"] = odf["id"]
deg_excerpt_2["channel"] = odf["channel"]
deg_excerpt_2["onset_sec"] = pt.utils.music.midi_ticks_to_seconds(deg_excerpt_1["onset"].to_numpy(), mpq=mpq, ppq=ppq)
deg_excerpt_2["duration_sec"] = pt.utils.music.midi_ticks_to_seconds(deg_excerpt_1["dur"].to_numpy(), mpq=mpq, ppq=ppq)
deg_excerpt_2.head()

In [44]:
records = deg_excerpt_1.to_records(index=False)
new_note_array = np.array(records, dtype = records.dtype.descr)
ppart = pt.performance.PerformedPart.from_note_array(new_note_array)
pt.save_performance_midi(ppart, "./artifacts/mdtk/piece_name.mid")